In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# import data

# 大表( 實價登錄 + 周邊設施 + 交通 )
preDF = pd.read_csv("./combine/presale_location_spot_MRT_bus.csv")
# 經濟因素資料
intDF = pd.read_csv("./data/eco_int.csv")

In [ ]:
# Select columns for variables
selectCol = ["鄉鎮市區", "土地區段位置建物區段門牌", "交易標的", "都市土地使用分區", "主要用途", "交易年月日", "總坪數", "每坪單價", "移轉層次", "建物現況格局-房", "建物現況格局-廳", "建物現況格局-衛", "bad100m", "edu500m", "med1000m", "class_MRT", "class_bus", "building_station_distance", "building_busstop_distance"]
preDF1 = preDF[selectCol]

# 要分析自住房屋，篩選出"住家"或"住商"
preDF2 = preDF1[(preDF1["主要用途"] == "住家用") | (preDF1["主要用途"] == "住商用")]


# 交易標的 => 房地(土地+建物) = 1、房地(土地+建物)+車位 = 2
def b_classify(row):
    if row["交易標的"] == "房地(土地+建物)":
        return 1
    elif row["交易標的"] == "房地(土地+建物)+車位":
        return 2
    else:
        return 3
    
# 都市土地使用分區 => 住 = 1、商 = 2、其他 = 3
def l_classify(row):
    if row["都市土地使用分區"] == "住":
        return 1
    elif row["都市土地使用分區"] == "商":
        return 2
    else:
        return 3

# 新增分類好的 "交易標的"、"土地使用" 欄位
preDF2["交易標的new"] = preDF2.apply(lambda x: b_classify(x), axis=1)
preDF2["土地使用"] = preDF2.apply(lambda x: l_classify(x), axis=1)

# 刪除不需要的欄位
preDF3 = preDF2.drop(["交易標的", "都市土地使用分區", "主要用途"], axis=1)

# "交易年月日" 整理為 "交易年月"
preDF3["交易年月日"] = preDF3["交易年月日"].astype(str)
preDF3["交易年月"] = preDF3["交易年月日"].str[0:5].astype(int)

# km 換成 m
preDF3["building_station_distance"] = preDF3["building_station_distance"]*1000
preDF3["building_busstop_distance"] = preDF3["building_busstop_distance"]*1000

# Rename columns
preDF4 = preDF3.rename(columns={"總坪數": "Area", "每坪單價": "Price", "移轉層次": "Floor", "建物現況格局-房": "Rooms", "建物現況格局-廳": "Liv_Din", "建物現況格局-衛": "Bath", "交易標的new": "Parking", "土地使用": "Landusage"})

# delete temporary objects
del preDF1, preDF2, preDF3

In [ ]:
# 合併兩張表
df2 = pd.merge(preDF4, intDF, left_on = "交易年月", right_on = "年月", how = "left").drop("年月", axis=1)

In [ ]:
df2.to_csv("./combine/模型用/combine.csv", index = False)